<a href="https://colab.research.google.com/github/brunobraganca/etl-generative-ai-challenge/blob/main/etl_generative_ai_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## etl-generative-ai-challenge

# **etl-generative-ai-challenge**
Python based challenge involving ETL pipeline with generative AI based on [SantanderDevWeek2023](https://github.com/digitalinnovationone/santander-dev-week-2023-api)

## Fantasy Context
A modern soccer courts venue called "Futebas" is currently working on new and technologic ways to engage with its costumers.

## Problem
Numbers of court reservations and income are falling. Marketing has decided to reach each and every costumer to get them back to the venue. According to "Futebas" guidelines, costumer approach should always focus on a friendly and mostly personal/custom form of communication.

## Proposition
Using Futebas CRM data, Python and OpenAi API, generate a customized SMS text as a call-to-action to reacquire the costumers.

# **Git Setup**


In [100]:
!git config --global user.name "USER"
!git config --global user.email "EMAIL"
!git config --global user.password "PASSWORD"

In [107]:
# Attention! Personal access token, not SSH
token = "TOKEN"
username = "USER"
repo = "REPO"

In [102]:
!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'etl-generative-ai-challenge'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (21/21), 8.89 KiB | 8.89 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [103]:
%cd {repo}

/content/etl-generative-ai-challenge


In [105]:
%ls

etl_generative_ai_challenge.ipynb  futebasCRM.csv      README.md
futebas_CRM.csv                    FutebasCrmData.csv


## **E**xtract

Extract users list.

In [115]:
import pandas as pd

crm_csv = pd.read_csv('/content/etl-generative-ai-challenge/futebasCRM.csv')
print(crm_csv)

      name  age     gender position         team   visited
0   Fabita   43   Feminino  Goleiro       Santos   3 meses
1  Rodrigo   18  Masculino   Ataque  Corinthians  10 meses
2   Flavio   59  Masculino   Defesa    São Paulo   15 dias
3  Daniela   35   Feminino   Ataque    Palmeiras   6 meses
4  Juliana   25   Feminino   Defesa  Corinthians   2 meses
5    Marco   39  Masculino  Goleiro  Corinthians   5 meses


In [77]:
import json
import io

usersString = crm_csv.to_json(orient='records')
users = json.loads(usersString)

print(users)

[{'name': 'Fabita', 'age': 43, 'gender': 'Feminino', 'position': 'Goleiro', 'team': 'Santos', 'visited': '3 meses'}, {'name': 'Rodrigo', 'age': 18, 'gender': 'Masculino', 'position': 'Ataque', 'team': 'Corinthians', 'visited': '10 meses'}, {'name': 'Flavio', 'age': 59, 'gender': 'Masculino', 'position': 'Defesa', 'team': 'São Paulo', 'visited': '15 dias'}, {'name': 'Daniela', 'age': 35, 'gender': 'Feminino', 'position': 'Ataque', 'team': 'Palmeiras', 'visited': '6 meses'}, {'name': 'Juliana', 'age': 25, 'gender': 'Feminino', 'position': 'Defesa', 'team': 'Corinthians', 'visited': '2 meses'}, {'name': 'Marco', 'age': 39, 'gender': 'Masculino', 'position': 'Goleiro', 'team': 'Corinthians', 'visited': '5 meses'}]


## **T**ransform

Transform user data throught OpenAI GPT-4 data to generate new SMS message.

In [38]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.1 MB/s eta 0:00:00


In [90]:
# https://platform.openai.com/account/api-keys

openai_api_key = 'TODO'

In [92]:
import openai

openai.api_key = openai_api_key

def generate_ai_sms(user):

  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[
      {
          "role": "system",
          "content": "Você é um boleiro profissa!"
      },
      {
          "role": "user",
          "content": f"Texto de mensagem SMS para a cliente {user['name']}, de {user['age']} anos, gênero {user['gender']}, torcedor(a) do {user['team']} e usuário(a) da quadra de futebol Futebas e que não retorna ao estabelecimento há {user['visited']}. (máximo de 200 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  msg = generate_ai_sms(user)
  print(msg)
  user['msg'] = msg

Olá Fabita, sentimos sua falta aqui na Futebas! A quadra e seus amigos te aguardam para jogar aquele futebol. Venha aproveitar nossos horários especiais. Santos sempre! 🐳⚽️
E aí Rodrigo, tudo bem? Sentimos sua falta aqui na Futebas! Que tal marcar uma partida com os amigos e matar a saudade da nossa quadra? Torcendo pelo Corinthians juntos!
Olá Flavio, tudo bem? Sentimos sua falta na Futebas! Esperamos que você esteja bem. Estamos com saudades dos seus jogos de tirar o fôlego. Volte logo! ⚽️👊 São Paulo sempre forte!
Olá Daniela! Sentimos sua falta na Futebas! Venha praticar seu futebol novamente e sentir a emoção em jogar em nossa quadra. Seu time e nós estamos esperando 👍🏼⚽. #AvantiPalmeiras.
Olá Juliana, notamos sua ausência na Futebas há 2 meses. Gostaríamos de saber se está tudo bem e se podemos ajudar de alguma forma. Esperamos vê-la em breve por aqui. Saudações corinthianas! 🏆⚽👍
Olá Marco! Sentimos sua falta aqui na Futebas, como torcedor fiel do Corinthians, sabemos que você é u

## **L**oad

Update and commit messages into new file.

In [124]:
usersJSON = pd.DataFrame(users)
usersJSON.to_csv("/content/etl-generative-ai-challenge/usersSMS.csv", index=False)


In [128]:
!git add --all

In [129]:
!git commit -a -m "Updated messages"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [130]:
!git push origin main

Everything up-to-date
